<a href="https://colab.research.google.com/github/whelfjhte79/BlindDateApp/blob/main/ideal_type_recommendation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/dataset/img.zip -d /content/img/

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
image = cv2.imread('/content/img/img500/1 (1).jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [ ]:
face_image = []
imgPath = "/content/img/"
for i in os.listdir(imgPath):
    pic = os.path.join(imgPath + "/", i)
    face_image.append(pic)

In [ ]:
os.listdir(imgPath)[:10]

In [ ]:
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(rescale=1./255)


In [ ]:
image_data = data_generator.flow_from_directory(
    '/content/img/',
    target_size=(200, 200),
    batch_size=1,
    class_mode='categorical')

In [ ]:
import pandas as pd
file_name = '/content/drive/MyDrive/dataset/face_data.csv'
csv_data = pd.read_csv(file_name)

In [ ]:
print(len(image_data))
print(len(image_data[0]))
print(len(image_data[0][0]))
print(len(image_data[0][0][0]))
print(len(image_data[0][0][0][0]))
print(len(image_data[0][0][0][0][0]))

In [ ]:
image_redution = [[[]]]   
for i in range(500):               # 이미지 5차원에서 3차원으로 수정  image_data -> image_redution
    image_redution.insert(i,image_data[i][0][0])

In [ ]:
plt.imshow(image_redution[499])   # image_redution 배열 축소 확인
plt.show()

In [ ]:
input_train = image_redution[0:400]        # X_train, X_test
input_test = image_redution[400:500]

X_train = np.array(input_train)
X_test = np.array(input_test)

print(X_train.shape)
print(X_test.shape)

In [ ]:
# mapping 한번만 실행
eyebrow_mapping = { "Thick": 1, "Normal": 2, "Thin": 3}
csv_data['Eyebrow'] = csv_data['Eyebrow'].map(eyebrow_mapping)
csv_data['Eyebrow'] = csv_data['Eyebrow'].fillna(0)
forehead_mapping = {"F": 1, "T": 2}
csv_data['Forehead'] = csv_data['Forehead'].map(forehead_mapping)

hairstyle_mapping = {"Skinhead": 1, "Upstyle": 2, "Long": 3, "Short": 4, "Perm": 5, "DownStyle": 6}
csv_data['Hairstyle'] = csv_data['Hairstyle'].map(hairstyle_mapping)

beard_mapping = {"F": 1, "T": 2}
csv_data['Beard'] = csv_data['Beard'].map(beard_mapping)

light_mapping = {"Black": 1, "Yellow": 2, "White": 3}
csv_data['Light'] = csv_data['Light'].map(light_mapping)

glasses_mapping = {"F": 1, "T": 2}
csv_data['Glasses'] = csv_data['Glasses'].map(glasses_mapping)

double_eyelid_mapping = {"F": 1, "T": 2}
csv_data['Double eyelid'] = csv_data['Double eyelid'].map(double_eyelid_mapping)

hat_mapping = {"F": 1, "T": 2}
csv_data['Hat'] = csv_data['Hat'].map(hat_mapping)

csv_data.head()
# 매칭
#eyebrow_mapping = { "Thick": 1, "Normal:": 2, "Thin": 3}
#csv_data['Eyebrow'] = csv_data['Eyebrow'].map(eyebrow_mapping)

In [ ]:
Y_train = csv_data.iloc[0:400]
Y_test = csv_data.iloc[400:500]

In [ ]:
# y값 합치기 or 다중 출력
# conv -> relu -> conv -> relu -> pool -> softmax

print(Y_train.shape)
print(Y_test.shape)


In [ ]:
from tensorflow import keras
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop,Adam

#model = keras.Sequential()

# 3.1 Conv2D layer(컨볼루션 레이어)
#model.add(Conv2D(filters = 5, 
#                 kernel_size = (3,3), # 1x1 or 3x3 디테일한 부분을 잡아야하니까
#                 padding = 'Same', 
#                 activation ='relu', 
#                 input_shape = (200,200,3)))
#model.add(MaxPool2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

In [ ]:
#placeholder 대체  1버전 -> 2버전

#tensorflow 1.x 
#self._states = tf.placeholder(shape=[None, self._num_states], dtype=tf.float32) 

#tensorflow 2.x 
#self._states = tf.Variable(tf.ones(shape=[None, self._num_states]), dtype=tf.float32)


In [ ]:
#define the convnet
model = keras.Sequential()
# CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(32, (3, 3), padding='same',input_shape = (200,200,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# CONV => RELU => CONV => RELU => POOL => DROPOUT
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# FLATTERN => DENSE => RELU => DROPOUT
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# a softmax classifier
model.add(Dense(8))
model.add(Activation('softmax'))

model.summary()

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train['Eyebrow'], epochs=50, batch_size=1)

In [ ]:
scores = model.evaluate(X_test, Y_test['Eyebrow'], verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
# accuracy 1.0000 인거 해결하고,
# validation 하고, 실행할때 GPU로 실행하고,
# Eyebrow,,,,, 9개 각각 실행할 지 합칠지 고민하고,
# model.fit 오래걸리니까 weight h5으로 저장하기